# Import Pandas and Numpy

In [65]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

# Parsing HTML Source Code to Form the Required Table Using lxml, requests, and Beautiful Soup

In [66]:
# Creating DataFrame for the table found in the URL below
from lxml import html
import requests
from bs4 import BeautifulSoup



# Getting a request from the URL and making a BeautifulSoup object for the HTML source code
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(page.text, 'html.parser')

# Creating BeautifulSoup objects for the table for the DataFrame
soup_table = soup.find("table", class_="wikitable sortable").find_all('tr')

soup_header = soup_table[0].find_all('th')

# Creating a table for the DataFrame
table = np.empty((len(soup_table), len(soup_header))).astype('str')


for row_index, row in enumerate(soup_table):
    for column_index, column in enumerate(soup_header):
        table[row_index] = list(text for text in soup_table[row_index].stripped_strings)
        
table = table.transpose()
        
# Creating the DataFrame
df = pd.DataFrame({table[0][0]: table[0][1:],
                   table[1][0]: table[1][1:],
                   table[2][0]: table[2][1:]})
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Removing Rows from Table with 'Not assigned' Values in 'Borough' Column

In [67]:
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df = df.reset_index(drop=True)
df.head(11)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


# Showing that there are NO Duplicate 'Postal Code' Values, but there ARE Duplicate 'Neighborhood' Values, but NO 'Not assigned' 'Neighborhood' Values

In [68]:
# This code shows that there are no duplicates in Postal Code and so we do not need to group different neighborhoods of the same Postal Code
print('Number of rows: ' + str(df.shape[0]))
print('Number of unique Postal Code labels: ' + str(df.groupby('Postal Code').ngroups))

# However, there are duplicate Neighborhoods, so we need to be sure to group by both the Postal Code and Neighborhood when trying to group by Neighborhoods.
print('Number of unique Neighborhood labels: ' + str(df.groupby('Neighborhood').ngroups))

# This code shows that there are no 'Not assigned' values in the Neighborhood column and so we do not need to assign these to any value in the Borough column
df.loc[df['Neighborhood'] == 'Not assigned']

Number of rows: 103
Number of unique Postal Code labels: 103
Number of unique Neighborhood labels: 99


,Postal Code,Borough,Neighborhood


# Getting Latitude/Longitude Data from given csv File since geocoder was not working

In [69]:
# Read lat, lng data from csv file since geocoder was not functioning
lat_lng_df = pd.read_csv('http://cocl.us/Geospatial_data')

toronto_data = df.merge(lat_lng_df)
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial",43.662301,-79.389494


# Keeping only the Data for Boroughs that contain 'Toronto' in their Name

In [70]:
# Keep only data for the Boroughs that end in 'Toronto'.
toronto_data = toronto_data[toronto_data['Borough'].str.contains("Toronto")]
toronto_data = toronto_data.reset_index(drop=True)
toronto_data.head(11)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


# Checking for Duplicate Neighborhoods

In [71]:
# This code shows that there are no duplicates in Postal Code and so we do not need to group different neighborhoods of the same Postal Code
print('Number of rows: ' + str(toronto_data.shape[0]))
print('Number of unique Postal Code labels: ' + str(toronto_data.groupby('Postal Code').ngroups))

# However, there are duplicate Neighborhoods, so we need to be sure to group by both the Postal Code and Neighborhood when trying to group by Neighborhoods.
print('Number of unique Neighborhood labels: ' + str(toronto_data.groupby('Neighborhood').ngroups))


Number of rows: 39
Number of unique Postal Code labels: 39
Number of unique Neighborhood labels: 39


# Import Folium and other Libraries

In [72]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



# Creating a Map of the Unsclustered Neighborhoods in Data using Folium

In [73]:
# create map of Manhattan using latitude and longitude values
latitude = 43.6532
longitude = -79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Import Json and set pandas options for max columns and rows
# Foursquare Credentials and radius and LIMIT variables for search queries. radius is the physical distance in meters around a particular latitude, longitude point and LIMIT details the maximum number of venues to be kept within this radius.

In [75]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

CLIENT_ID = '5A2XXCBQWE1YNEFR5B1FIR1LQ40EWRKA1YHTZO3GGBS3UFFI' # your Foursquare ID
CLIENT_SECRET = 'IHFJIG4OIPNF1PEL2NHQTT3DPY5RLX250V0LDQGB4MBO1F0K' # your Foursquare Secret
VERSION = '20200602' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

radius = 500
LIMIT = 100


Your credentails:
CLIENT_ID: 5A2XXCBQWE1YNEFR5B1FIR1LQ40EWRKA1YHTZO3GGBS3UFFI
CLIENT_SECRET:IHFJIG4OIPNF1PEL2NHQTT3DPY5RLX250V0LDQGB4MBO1F0K


# Creating a function to Get the Nearby Venues for the Neighborhoods using requests on the json file from Foursquare API.
# Output is a DataFrame with the Neighborhood, Neighborhood Latitude, Neighbhoord Longitude, Venue Name, Venue Latitude, Venue Longitude, and Venue Category as columns and each row will be one of the venues for each Neighborhood 
# We will use k-means clustering based on the top 10 Venue Categories for each Neighbhoord, as was done in the lab for New York

In [76]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Running the above function for our Toronto Neighborhood Data

In [77]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )
toronto_venues.head()
print(toronto_venues.shape)

(1612, 7)


# Using One Hot Encoding on the Venue Categories to find the Mean of Venue Categories for each Neighborhood that will be used for k-means

In [78]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
#toronto_onehot = toronto_onehot[fixed_columns]
first_col = toronto_onehot.pop('Neighborhood')
toronto_onehot.insert(0, 'Neighborhood', first_col)

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,

# Finding the Mean of Venue Categories over all the Venues for each Neighborhood

In [79]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped
toronto_grouped.shape

(39, 236)

# Creating a function that will return the Most Common Venues in the form of a list based on the Above One Hot Data

In [80]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# Creating a DataFrame for the Most Common Venues for each Neighborhood based on the above function

In [81]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Pub,Café,Cheese Shop,Bakery,Restaurant,Beer Bar,Seafood Restaurant,Clothing Store
1,"Brockton, Parkdale Village, Exhi",Café,Coffee Shop,Breakfast Spot,Pet Store,Italian Restaurant,Stadium,Restaurant,Intersection,Bar,Climbing Gym
2,Business reply mail Processing C,Yoga Studio,Auto Workshop,Comic Shop,Pizza Place,Restaurant,Butcher,Burrito Place,Skate Park,Brewery,Farmers Market
3,"CN Tower, King and Spadina, Rail",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Airport,Airport Food Court,Bar,Sculpture Garden,Boutique,Rental Car Location
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Chinese Restaurant,Department Store,Bubble Tea Shop,Burger Joint,Salad Place


# Import k-means
# Using k-means with k=5 clusters on the above DataFrame

In [82]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

# Adding a column 'Cluster Labels' that identifies the k-means cluster of each Neighborhood

In [83]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(-1) # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Pub,Park,Café,Breakfast Spot,Theater,Ice Cream Shop,Mexican Restaurant,Chocolate Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Fried Chicken Joint,Sandwich Place,Café,Restaurant,Smoothie Shop,Italian Restaurant,Distribution Center,Beer Bar
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Italian Restaurant,Japanese Restaurant,Tea Room,Bookstore
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Cocktail Bar,Restaurant,American Restaurant,Gastropub,Gym,Cosmetics Shop,Creperie,Moroccan Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Pub,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Pub,Café,Cheese Shop,Bakery,Restaurant,Beer Bar,Seafood Restaurant,Clothing Store
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Chinese Restaurant,Department Store,Bubble Tea Shop,Burger Joint,Salad Place
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Grocery Store,Café,Park,Baby Store,Bank,Diner,Restaurant,Italian Restaurant,Nightclub,Coffee Shop
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,1,Coffee Shop,Café,Restaurant,Hotel,Gym,Deli / Bodega,Clothing Store,Thai Restaurant,Concert Hall,Steakhouse
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1,Pharmacy,Bakery,Grocery Store,Music Venue,Middle Eastern Restaurant,Café,Brewery,Supermarket,Bar,Bank


# Using Folium to show a map of the clustered Neighborhoods

In [84]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels'].astype('int')):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# According to the map, the largest cluster has neighborhoods near the central part of Toronto and the other clusters seemed to be correlated to their neighborhood's distances from the central part of Toronto.

In [85]:
print('Cluster 0')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].sort_values(by='Borough')

Cluster 0


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Central Toronto,0,Coffee Shop,Pub,Liquor Store,Sports Bar,Restaurant,Bank,Supermarket,Sushi Restaurant,Fried Chicken Joint,Bagel Shop
4,East Toronto,0,Trail,Health Food Store,Pub,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio


In [86]:
print('Cluster 1')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].sort_values(by='Borough')

Cluster 1


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Central Toronto,1,Sandwich Place,Dessert Shop,Coffee Shop,Pizza Place,Café,Italian Restaurant,Sushi Restaurant,Gym,Greek Restaurant,Gourmet Shop
24,Central Toronto,1,Café,Sandwich Place,Coffee Shop,Pizza Place,Middle Eastern Restaurant,Indian Restaurant,Pub,Donut Shop,BBQ Joint,History Museum
23,Central Toronto,1,Coffee Shop,Clothing Store,Yoga Studio,Bagel Shop,Spa,Fast Food Restaurant,Sporting Goods Shop,Diner,Metro Station,Salon / Barbershop
20,Central Toronto,1,Park,Hotel,Food & Drink Shop,Sandwich Place,Department Store,Dance Studio,Breakfast Spot,Pizza Place,Gym / Fitness Center,Convenience Store
0,Downtown Toronto,1,Coffee Shop,Bakery,Pub,Park,Café,Breakfast Spot,Theater,Ice Cream Shop,Mexican Restaurant,Chocolate Shop
36,Downtown Toronto,1,Coffee Shop,Café,Gym,Restaurant,Hotel,Japanese Restaurant,Seafood Restaurant,Salad Place,Deli / Bodega,American Restaurant
35,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Bakery,Pizza Place,Chinese Restaurant,Café,Restaurant,Pub,Liquor Store,Snack Place
34,Downtown Toronto,1,Coffee Shop,Café,Beer Bar,Pub,Seafood Restaurant,Restaurant,Japanese Restaurant,Italian Restaurant,Cocktail Bar,Breakfast Spot
32,Downtown Toronto,1,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Airport,Airport Food Court,Bar,Sculpture Garden,Boutique,Rental Car Location
30,Downtown Toronto,1,Café,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Bakery,Vegetarian / Vegan Restaurant,Park,Bar,Dessert Shop,Gaming Cafe


In [87]:
print('Cluster 2')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 2


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,2,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
29,Central Toronto,2,Park,Playground,Restaurant,Tennis Court,Yoga Studio,Distribution Center,Dessert Shop,Diner,Discount Store,Doner Restaurant
33,Downtown Toronto,2,Park,Playground,Trail,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop


In [88]:
print('Cluster 3')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].sort_values(by='Borough')

Cluster 3


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Park,Bus Line,Swim School,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop


In [89]:
print('Cluster 4')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].sort_values(by='Borough')

Cluster 4


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,4,Pool,Ice Cream Shop,Garden,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop


# An initial exmaniation of the clusters shows that the most common venues of coffee shops, cafes, and parks seem to be influential features in the clustering. Cluster 4 has little in common with the other clusters, with pool as the most common venue. It seems that outside venues help define the smaller clusters, which makes sense with our observations from the map that clusters are related to their distance from the central part of Toronto.